Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data {
  int N;
  int<lower=0, upper=1> L[N];
  vector[N] P;
  vector[N] C;

  int<lower=1, upper=N> N_chimps;
  int<lower=1, upper=N_chimps> chimp[N];
}
parameters {
  real a_chimp[N_chimps];
  real bp;
  real bpc;
}
model {
  vector[N] p;
  target += normal_lpdf(a_chimp | 0, 10);
  target += normal_lpdf(bp | 0, 10);
  target += normal_lpdf(bpc | 0, 10);
  for (i in 1:N) p[i] = a_chimp[chimp[i]] + (bp + bpc * C[i]) * P[i];
  target += binomial_logit_lpmf(L | 1, p);
}
generated quantities {
  vector[N] log_lik;
  {
    vector[N] p;
    for(n in 1:N) {
      p[n] = a_chimp[chimp[n]] + (bp + bpc * C[n]) * P[n];
      log_lik[n] = binomial_logit_lpmf(L[n] | 1, p[n]);
    }
  }
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/notebooks/10/tmp/m_10_04.stan will be updated.



Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_chimps" => length(unique(df[:actor])),
"chimp" => df[:actor], "L" => df[:pulled_left],
"P" => df[:prosoc_left], "C" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpc"],
  :pooled => ["a_chimp.$i" for i in 1:7],
  :generated => ["log_lik.$i" for i in 1:504],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×520×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
generated         = log_lik.1, log_lik.2, log_lik.3, log_lik.4, log_lik.5, log_lik.6, log_lik.7, log_lik.8, log_lik.9, log_lik.10, log_lik.11, log_lik.12, log_lik.13, log_lik.14, log_lik.15, log_lik.16, log_lik.17, log_lik.18, log_lik.19, log_lik.20, log_lik.21, log_lik.22, log_lik.23, log_lik.24, log_lik.25, log_lik.26, log_lik.27, log_lik.28, log_lik.29, log_lik.30, log_lik.31, log_lik.32, log_lik.33, log_lik.34, log_lik.35, log_lik.36, log_lik.37, log_lik.38, log_lik.39, log_lik.40, log_lik.41, log_lik.42, log_lik.43, log_lik.44, log_lik.45, log_lik.46, log_lik.47, log_lik.48, log_lik.49, log_lik.50, log_lik.51, log_lik.52, log_lik.53, log_lik.54, log_lik.55, log_lik.56, log_lik.57, log_lik.58, log_lik.59, log_lik.60, log_lik.61, log_lik.62, log_lik.63, log_lik.64, log_lik.65, log_lik.66, log_lik.67, log_lik.68

Describe parameter draws

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = bp, bpc

Empirical Posterior Estimates
─────────────────────────────────────────
parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8356 0.2624   0.0041 0.0053 1000
bpc -0.1267 0.3045   0.0048 0.0056 1000

Quantiles
─────────────────────────────────────────
parameters
      2.5%   25.0%   50.0%   75.0%  97.5%
 bp  0.3213  0.6607  0.8337 1.0130 1.3361
bpc -0.7368 -0.3262 -0.1286 0.0846 0.4510



Describe pooled parameter draws

In [10]:
describe(chn2, section=:pooled)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a_chimp.1, a_chimp.2, a_chimp.3, a_chimp.4, a_chimp.5, a_chimp.6, a_chimp.7

Empirical Posterior Estimates
─────────────────────────────────────────────────
pooled
            Mean    SD   Naive SE  MCSE   ESS
a_chimp.1 -0.7367 0.2714   0.0043 0.0039 1000
a_chimp.2 10.8363 5.2378   0.0828 0.1365 1000
a_chimp.3 -1.0509 0.2764   0.0044 0.0052 1000
a_chimp.4 -1.0563 0.2837   0.0045 0.0050 1000
a_chimp.5 -0.7392 0.2694   0.0043 0.0046 1000
a_chimp.6  0.2189 0.2679   0.0042 0.0042 1000
a_chimp.7  1.8042 0.4006   0.0063 0.0074 1000

Quantiles
─────────────────────────────────────────────────
pooled
            2.5%   25.0%   50.0%   75.0%   97.5% 
a_chimp.1 -1.2647 -0.9267 -0.7342 -0.5482 -0.2181
a_chimp.2  3.9724  6.7996  9.8008 13.6871 23.4016
a_chimp.3 -1.6146 -1.2359 -1.0389 -0.8570 -0.5272
a_chimp.4 -1.6248 -1.2480 -1.0554 -0.8613 -0.5221
a_chimp.5 -1.2835 -0.9215

End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*